# Notebook Utils

This notebook contains helper functions and classes for use with OpenVINO™ Notebooks. The code is synchronized with the `notebook_utils.py` file in the same directory as this notebook.

There are five categories:

- [Files](#Files)
- [Images](#Images)
- [Videos](#Videos)
- [Visualization](#Visualization)
- [OpenVINO Tools](#OpenVINO-Tools)
- [Checks and Alerts](#Checks-and-Alerts)

Each category contains a test cell that also shows how to use the functions in the section. 

## Files

Load an image, download a file, download an OpenVINO IR model, and create a progress bar to show download progress.

In [ ]:
import os
import shutil
from PIL import Image
from notebook_utils import load_image, download_file, download_ir_model

In [ ]:
??load_image

In [ ]:
??download_file

In [ ]:
??download_ir_model

### Test File Functions

In [ ]:
model_url = "https://github.com/openvinotoolkit/openvino_notebooks/raw/main/notebooks/002-openvino-api/model/segmentation.xml"
download_ir_model(model_url, "model")

assert os.path.exists("model/segmentation.xml")
assert os.path.exists("model/segmentation.bin")

In [ ]:
url = "https://github.com/intel-iot-devkit/safety-gear-detector-python/raw/master/resources/Safety_Full_Hat_and_Vest.mp4"
if os.path.exists(os.path.basename(url)):
    os.remove(os.path.basename(url))
video_file = download_file(url)
print(video_file)
assert os.path.exists(video_file)

In [ ]:
url = "https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/README.md"
filename = "openvino_notebooks_readme.md"
if os.path.exists(filename):
    os.remove(filename)
readme_file = download_file(url, filename=filename)
print(readme_file)
assert os.path.exists(readme_file)

In [ ]:
url = "https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/README.md"
filename = "openvino_notebooks_readme.md"
directory = "temp"
video_file = download_file(
    url, filename=filename, directory=directory, show_progress=False, silent=True
)
print(readme_file)
assert os.path.exists(readme_file)
shutil.rmtree("temp")

In [ ]:
url = "../data/image/coco.jpg"
image = load_image(url)
Image.fromarray(image[:, :, ::-1])

## Images

### Convert Pixel Data

Normalize image pixel values between 0 and 1, and convert images to `RGB` and `BGR`.

In [ ]:
import numpy as np
from notebook_utils import normalize_minmax, to_rgb, to_bgr

In [ ]:
??normalize_minmax

In [ ]:
??to_bgr

In [ ]:
??to_rgb

### Test Data Conversion Functions

In [ ]:
test_array = np.random.randint(0, 255, (100, 100, 3))
normalized_array = normalize_minmax(test_array)

assert normalized_array.min() == 0
assert normalized_array.max() == 1

In [ ]:
bgr_array = np.ones((100, 100, 3), dtype=np.uint8)
bgr_array[:, :, 0] = 0
bgr_array[:, :, 1] = 1
bgr_array[:, :, 2] = 2
rgb_array = to_rgb(bgr_array)

assert np.all(bgr_array[:, :, 0] == rgb_array[:, :, 2])

bgr_array_converted = to_bgr(rgb_array)
assert np.all(bgr_array_converted == bgr_array)

## Videos

### Video Player

A custom video player to fulfill FPS requirements. You can set target FPS and output size, flip the video horizontally or skip first N frames.

In [ ]:
import cv2
from IPython.display import Image, clear_output, display
from notebook_utils import VideoPlayer

??VideoPlayer

### Test Video Player

In [ ]:
video = "../data/video/Coco Walking in Berkeley.mp4"

player = VideoPlayer(video, fps=15, skip_first_frames=10)
player.start()
for i in range(50):
    frame = player.next()
    _, encoded_img = cv2.imencode(".jpg", frame, params=[cv2.IMWRITE_JPEG_QUALITY, 90])
    img = Image(data=encoded_img)
    clear_output(wait=True)
    display(img)

player.stop()
print("Finished")

## Visualization

### Segmentation

Define a `SegmentationMap NamedTuple` that keeps the labels and colormap for a segmentation project/dataset. Create `CityScapesSegmentation` and `BinarySegmentation SegmentationMaps`. Create a function to convert a segmentation map to an `RGB` image with a `colormap`, and to show the segmentation result as an overlay over the original image.

In [ ]:
from notebook_utils import SegmentationMap, CityScapesSegmentation, BinarySegmentation, segmentation_map_to_image, segmentation_map_to_overlay

In [ ]:
??Label

In [ ]:
??SegmentationMap

In [ ]:
??CityScapesSegmentation

In [ ]:
print(f"cityscapes segmentation lables: \n{CityScapesSegmentation.get_labels()}")
print(f"cityscales segmentation colors: \n{CityScapesSegmentation.get_colormap()}")

In [ ]:
??BinarySegmentation

In [ ]:
print(f"binary segmentation lables: \n{BinarySegmentation.get_labels()}")
print(f"binary segmentation colors: \n{BinarySegmentation.get_colormap()}")

In [ ]:
??segmentation_map_to_image

In [ ]:
??segmentation_map_to_overlay

### Network Results

Show network result image, optionally together with the source image and a legend with labels.

In [ ]:
from notebook_utils import viz_result_image

??viz_result_image

### Test Visualization Functions

In [ ]:
testimage = np.zeros((100, 100, 3), dtype=np.uint8)
testimage[30:80, 30:80, :] = [0, 255, 0]
testimage[0:10, 0:10, :] = 100
testimage[40:60, 40:60, :] = 128
testimage[testimage == 0] = 128


testmask1 = np.zeros((testimage.shape[:2]))
testmask1[30:80, 30:80] = 1
testmask1[40:50, 40:50] = 0
testmask1[0:15, 0:10] = 2

result_image_overlay = segmentation_map_to_overlay(
    image=testimage,
    result=testmask1,
    alpha=0.6,
    colormap=np.array([[0, 0, 0], [255, 0, 0], [255, 255, 0]]),
)
result_image = segmentation_map_to_image(testmask1, CityScapesSegmentation.get_colormap())
result_image_no_holes = segmentation_map_to_image(
    testmask1, CityScapesSegmentation.get_colormap(), remove_holes=True
)
resized_result_image = cv2.resize(result_image, (50, 50))
overlay_result_image = segmentation_map_to_overlay(
    testimage, testmask1, 0.6, CityScapesSegmentation.get_colormap(), remove_holes=False
)

fig1 = viz_result_image(result_image, testimage)
fig2 = viz_result_image(result_image_no_holes, testimage, labels=CityScapesSegmentation)
fig3 = viz_result_image(
    resized_result_image,
    testimage,
    source_title="Source Image",
    result_title="Resized Result Image",
    resize=True,
)
fig4 = viz_result_image(
    overlay_result_image,
    labels=CityScapesSegmentation,
    result_title="Image with Result Overlay",
)

display(fig1, fig2, fig3, fig4)

### Live Inference

In [ ]:
from notebook_utils import show_live_inference

In [ ]:
??show_live_inference

#### Test Live Inference

In [ ]:
# Test binary segmentation
from models.custom_segmentation import SegmentationModel
from pathlib import Path
from openvino.runtime import Core

image_paths = sorted(list(Path("../111-detection-quantization/data").glob("*.jpg")))

ie = Core()
segmentation_model = SegmentationModel(
    ie,
    Path("model/segmentation.xml"),
    sigmoid=False,
    colormap=np.array([[0, 0, 0], [0, 0, 255]]),
    rgb=True,
    rotate_and_flip=False,
)

show_live_inference(
    ie=ie,
    image_paths=image_paths,
    model=segmentation_model,
    device="CPU",
    reader=lambda x: cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB),
)

In [ ]:
# Test multiclass segmentation with different input shape.
# This requires running the 102 notebook first, to generate the Fastseg model.

fastseg_path = Path("../102-pytorch-onnx-to-openvino/model/fastseg1024.xml")
image_path = "../data/image/street.jpg"

if fastseg_path.exists():
    image_paths = [
        image_path,
    ] * 5

    ie = Core()
    CityScapesSegmentation = SegmentationMap(cityscape_labels)
    segmentation_model = SegmentationModel(
        ie,
        fastseg_path,
        sigmoid=False,
        argmax=True,
        colormap=CityScapesSegmentation.get_colormap(),
        rgb=True,
    )
    show_live_inference(ie=ie, image_paths=image_paths, model=segmentation_model, device="CPU")

## Checks and Alerts

Create an alert class to show stylized info/error/warning messages and a `check_device` function that checks whether a given device is available.

In [ ]:
from notebook_utils import NotebookAlert, DeviceNotFoundAlert, check_device, check_openvino_version

In [ ]:
??NotebookAlert

In [ ]:
??DeviceNotFoundAlert

In [ ]:
??check_device

In [ ]:
??check_openvino_version

### Test Alerts

In [ ]:
NotebookAlert(message="Hello, world!", alert_class="info")
DeviceNotFoundAlert("GPU");

In [ ]:
assert check_device("CPU")

In [ ]:
if check_device("HELLOWORLD"):
    print("Hello World device found.")

In [ ]:
check_openvino_version("2022.1");